# 🧠 HealthAI - Generative AI Health Assistant

In [ ]:
!pip install gradio transformers accelerate

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import gradio as gr
import os

In [ ]:
model_name = "ibm-granite/granite-3.3-2b-instruct"
hf_token = os.getenv("HF_TOKEN")  # Set this in your environment

tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=hf_token,
    device_map="auto",
    trust_remote_code=True
)

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

### 🔍 Function: Identify Disease

In [ ]:
def identify_disease(symptoms):
    prompt = f"Given the symptoms: {symptoms}, identify the possible disease."
    result = generator(prompt, max_new_tokens=100)[0]['generated_text']
    response = result.split(prompt)[-1].strip()
    return f"*Predicted Disease:*\n\n{response}"

### 🌿 Function: Suggest Home Remedies

In [ ]:
def home_remedies(disease):
    prompt = f"Suggest natural home remedies for the disease: {disease}."
    result = generator(prompt, max_new_tokens=100)[0]['generated_text']
    response = result.split(prompt)[-1].strip()
    return f"*Home Remedies for {disease.capitalize()}:*\n\n{response}"

### 🤖 Function: General Health Query

In [ ]:
def health_query(question):
    prompt = f"As a health assistant, answer the following question clearly:\nQuestion: {question}"
    result = generator(prompt, max_new_tokens=150)[0]['generated_text']
    response = result.split(prompt)[-1].strip()
    return f"*Answer:*\n\n{response}"

### 🚀 Launch Gradio App

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("# 🧠 HealthAI - Your Personal AI Health Assistant")

    with gr.Tab("🩺 Symptoms Identifier"):
        symptoms_input = gr.Textbox(label="Enter your symptoms")
        disease_output = gr.Textbox(label="Predicted Disease")
        symptoms_button = gr.Button("Predict Disease")
        symptoms_button.click(fn=identify_disease, inputs=symptoms_input, outputs=disease_output)

    with gr.Tab("🌿 Home Remedies"):
        disease_input = gr.Textbox(label="Enter Disease")
        remedy_output = gr.Textbox(label="Home Remedy")
        remedy_button = gr.Button("Get Remedy")
        remedy_button.click(fn=home_remedies, inputs=disease_input, outputs=remedy_output)

    with gr.Tab("🤖 Ask a Health Question"):
        user_question = gr.Textbox(label="Ask a health-related question", placeholder="e.g., How to prevent diabetes?")
        answer_output = gr.Textbox(label="AI Answer")
        ask_button = gr.Button("Get Answer")
        ask_button.click(fn=health_query, inputs=user_question, outputs=answer_output)

demo.launch()